## Flipping the order: matching *before* propensity score
Most observational studies of causal effects that compare treated people with unexposed people have 4 main sets of choices:

1. Gather the treated people and a comparable never-treated population, such as people who never took the drug of interest, but are taking a drug with a similar indication.
2. Calculate propensity score to quantify the dependence between pre-treatment covariates and treatment.
3. Create a matched population:  **a**) matching treated and controls on the propensity score; or **b**) less commonly, do Inverse Probability of Treatment weighting.
4. Evaluate the matching by comparing the pre-treatment covariates.

Of these steps, I want to to keep step (2) and (3b) the same but change 1 and 4:
1. Learn a roughly matched population
2. Same as above
3. Implement part (b) to create weighted comparison pseudopopulations
4. Evaluate the population matching by looking at the effect estimates

## Current work:  Step 1-- learn a roughly matched population
### Why?
We want to do something analagous to a manual selection of a comparison population, but without researching each drug individually to find a good comparison cohort of untreated people. 

Although Step 2 is designed to make the populations exactly comparable, it will work better if they are already roughly comparable.  Say that we have a drug for glaucoma, and thus everyone on the drug has a history of glaucoma; but among people not on the drug, only < 1% have glaucoma.  If we just pick randomly from all untreated people, then most of them will not have glaucoma, and so the indicator of "has glaucoma" would form a very accurate predictor, but not a great way of making a comparable population.  Thus, I hope that we could learn to automatically select other glaucoma patients for the comparison group.

### How?
Say we sample a single year from each patient, represent it with a small dense vector (30-dimensional) containing:

- 20-dimensions of the disease embedding, summarizing the diagnosis history over the last year before the index date.  For patient $i$, let the list of $j$ Dx in the past year is $d_i$. The embedding matrix is $E$, with one row per disease. Then, this is $h_i = \sum_j E[d_{i,j},:] $
- Say 10 more for:
 - Age
 - Year
 - Gender
 - Number of Rx and number of Dx (and/or Suffering)
 - County (represented by the county's PCA projections from the "patchwork" paper)
 
Then, we want to find the similar patient-years to the Treated by comparing these vectors. 

#### My first thought: nearest neighbor in the dense space
What I want to avoid is going straight to step 2 and learning a propensity score and then trimming the population by the propensity score. That totally defeats the purpose of the propensity score, which is supposed to represent a distribution of treatment in a population.

Thus, my first thought was to do an unsupervised approach like K-nearest neighbors for each Treated patient.  However, this seems like a pretty difficult problem itself when you have many millions of points in your sample, and you want to find a matched set to millions of query points (treated patients), then 30 is still pretty high-dimensional. 

Another option would be to try to make an even smaller vector to represent the patients, but not sure if that is realistic.

#### Coming back to propensity score matching
We can learn a simple logistic regression model on the 30-dimensional vector, and match patients on the propensity scores.  This is only for step 1, of finding a comparison set of patients.  It still seems a little fishy to use the propensity score to make the comparison population and then another propensity score to determine the relationship between treatment selection and covariates. But, it's much easier to match on one variable, and it might have a similar result.

### Evaluating the roughly matched population
It should be easy to evaluate if we have a good roughly matched population by looking at which drug they have an incident prescription for:  we should see a lot of drugs in the same therapeutic class, or related therapeutic purposes. 